In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

sys.path.append('..//')
from utils_mitgcm import open_mitgcm_ds_from_config
from utils_energy_analysis import *

In [ ]:
model = 'geneva_dummy_extended'
mitgcm_config, ds = open_mitgcm_ds_from_config('..//config.json', model)

In [ ]:
output_folder = r'/home/leroquan@eawag.wroot.emp-eaw.ch/work_space/dummy_extended/energy_budget'

In [ ]:
grid_resolution = 200
ds['YC'] = np.arange(1, len(ds['YC']) + 1) * grid_resolution - grid_resolution / 2
ds['XC'] = np.arange(1, len(ds['XC']) + 1) * grid_resolution - grid_resolution / 2
ds['YG'] = np.arange(0, len(ds['YG'])) * grid_resolution
ds['XG'] = np.arange(0, len(ds['XG'])) * grid_resolution

In [ ]:
%matplotlib inline
t=-1
plt.close('all')
plt.figure(figsize=(15,7))
ds_sel = ds['THETA'].isel(Z=0, time=t)
ds_sel.where(ds_sel != 0, np.nan).plot()
plt.title(ds_sel.time.values)
#plt.savefig(r"/home/leroquan@eawag.wroot.emp-eaw.ch/work_space/dummy_extended/figures/transect_location.png")
plt.show()

In [ ]:
aligned_u = ds.UVEL.rename({'XG':'XC'})
aligned_u['XC'] = ds['XC']

aligned_v = ds.VVEL.rename({'YG':'YC'})
aligned_v['YC'] = ds['YC']

aligned_w = ds.WVEL.rename({'Zl':'Z'})
aligned_w['Z'] = ds['Z']

# KE whole lake

In [ ]:
ke_tot = compute_ke(
    aligned_u,
    aligned_v,
    aligned_w,
    200,
    200,
    ds.drF)

In [ ]:
ke_tot.sum(dim=['XC','YC','Z']).plot()

df_ke_tot = ke_tot.sum(dim=['XC','YC','Z']).to_dataframe(name='ke_mj_total')['ke_mj_total'].reset_index()

df_ke_tot.to_csv(os.path.join(output_folder, "ke_whole_lake.csv"))

# KE surface

In [ ]:
ke_surf = compute_ke(aligned_u.isel(Z=0), aligned_v.isel(Z=0), aligned_w.isel(Z=0), 200, 200, 0.5)

In [ ]:
ke_surf.isel(time=-1).plot()
plt.hlines(11000,0,70000,'red')

In [ ]:
ke_surf.isel(time=-1).sel(YC=11000, method='nearest').plot()

for i in range(ke_surf.sizes['time']):
    plt.close('all')
    plt.figure(figsize=(15,7))
    ke_surf.isel(time=i).plot(vmax=1)
    plt.title(ke_surf.time.isel(time=i).values)
    plt.savefig(rf"/home/leroquan@eawag.wroot.emp-eaw.ch/work_space/dummy_extended/seiche_analysis/map_total_ke_surface/ke_surface_time{i}.png")

In [ ]:
ke_surf.sum(dim=['XC','YC']).plot()

In [ ]:
total_ke_surf = ke_surf.sum(dim=['XC','YC'])

# Spectral analysis of total KE

In [ ]:
from utils_signal_processing import *

In [ ]:
output_folder = r"/home/leroquan@eawag.wroot.emp-eaw.ch/work_space/dummy_extended/seiche_analysis/ke_analysis"

In [ ]:
xc = 280
yc = 40
point_ke_surf = ke_surf.isel(XC=xc,YC=yc)

In [ ]:
point_ke_surf.plot()
plt.title(f'XC={xc}, YC={yc}')
plt.savefig(os.path.join(output_folder,f'timeserie_XC={xc}_YC={yc}.png'))

In [ ]:
ke_surf.isel(time=24).plot()
plt.scatter(point_ke_surf.XC.values,point_ke_surf.YC.values, c='red')
plt.savefig(os.path.join(output_folder,f'point_XC={xc}_YC={yc}.png'))

In [ ]:
ke_fft = xr_compute_meanfft(point_ke_surf.load(), M=1).compute()

In [ ]:
cutoff1_hr = 45
cutoff2_hr = 20

cutoff1 = 1/(cutoff1_hr * 3600)
cutoff2 = 1/(cutoff2_hr * 3600)

fig,ax = plot_freq_spectrum(ke_fft, 'U', depth=0, m_segm=1, y_lim_min=1e-9, x_lim_min=0.01e-4, fontsize=10)
fig.savefig(os.path.join(output_folder,f'spectrum_XC={xc}_YC={yc}.png'))
#ax.axvline(x = cutoff1, linestyle="--", color="k",label="cutoff1")
#ax.axvline(x = cutoff2, linestyle="--", color="k",label="cutoff2")

In [ ]:
ke_filtered = filter_signal_xarray(ke_tot, btype='bandpass', time_dim='time', dt=3600, period_cutoff_low=(cutoff1_hr*3600), period_cutoff_high=(cutoff2_hr*3600), order=5)

In [ ]:
ke_filtered.plot()
ke_tot.plot()
(ke_tot - ke_filtered).plot()